In [2]:
import urllib.request
import re

### 读取数据

In [3]:
url = ("https://raw.githubusercontent.com/rasbt/" 
 "LLMs-from-scratch/main/ch02/01_main-chapter-code/" 
 "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x21d99d72870>)

### 读取文档

In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number", len(raw_text))
print(raw_text[:99])

Total number 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### 构建vocabulary(按单词切分)

In [5]:
processed_text = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
processed_text = [item.strip() for item in processed_text if item.strip()]
print(len(processed_text))

4690


In [6]:
# 转化token id
all_words = sorted(set(processed_text))
vocab_size = len(all_words)
print(vocab_size)

1130


In [7]:
# 构建词汇表
vocab = {token:id for id, token in enumerate(all_words)}
for id, token in enumerate(vocab.items()) :
    print(token)
    if id > 5:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)


### 构建分词器

In [8]:
# 构建文本分词器
class SimpleTokenizer1:
    def __init__(self, vocab):
        self.id_to_token = {id:token for token, id in vocab.items()}
        self.token_to_id = vocab
    def encoder(self, text):
        processed_text = re.split(r'([,.?_!"()\']|--|\s)', text)
        processed_text = [item.strip() for item in processed_text if item.strip()]
        ids = [self.token_to_id[s] for s in processed_text]
        return ids
    def decoder(self, ids):
        text = " ".join([self.id_to_token[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [9]:
tokenizer = SimpleTokenizer1(vocab) 
text = """"It's the last he painted, you know," 
 Mrs. Gisburn said with pardonable pride.""" 
ids = tokenizer.encoder(text) 
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [10]:
print(tokenizer.decoder(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


### 引入特殊token（一般有bos，eos，pad，unk，endoftext）

In [11]:
all_tokens = sorted(list(set(processed_text)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:id for id, token in enumerate(all_tokens)}
print(len(vocab.items()))

1132


In [12]:
# 构建文本分词器
class SimpleTokenizer:
    def __init__(self, vocab):
        self.id_to_token = {id:token for token, id in vocab.items()}
        self.token_to_id = vocab
    def encoder(self, text):
        processed_text = re.split(r'([,.?_!"()\']|--|\s)', text)
        processed_text = [item.strip() for item in processed_text if item.strip()]
        processed_text = [token if token in self.token_to_id else "<unk>" for token in processed_text]
        ids = [self.token_to_id[token] for token in processed_text]
        return ids
    def decoder(self, ids):
        text = " ".join([self.id_to_token[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

## 不同Tokenizer算法的学习

#### BPE算法

In [16]:
# 使用
!pip install tiktoken
import tiktoken

   ---------------------------------------- 0.0/878.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/878.7 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/878.7 kB ? eta -:--:--
   ---------------------------------------- 878.7/878.7 kB 1.7 MB/s eta 0:00:00


In [17]:
tokenizer = tiktoken.get_encoding("gpt2")
text = ( 
 "Hello, do you like tea? <|endoftext|> In the sunlit terraces" 
 "of someunknownPlace." 
) 
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"}) 
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


##### BPE分词就是将词划分为子词，因为按照词分，词表过大，且无法解决OOV问题，按字符分无法把握语义结构，子词划分不仅词表较小，对OOV的词也能拆解成子词理解。

### 过程
#### 1. 结尾加上特殊字符_
#### 2. 统计相邻字符出现的频率
#### 3. 合并出现频率最高的相邻字符，将两个字符合并成一个重新加入
#### 4. 重复3， 4，直到达到预计的词表大小

### WordPiece算法

In [22]:
# 使用
!pip install transformers
from transformers import BertTokenizer

# 加载 BERT 的 WordPiece 分词器
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 示例文本
text = "Natural language processing is fun!"

# 编码（分词）
tokens = tokenizer.tokenize(text)
print("Tokens:", tokens)

# 将 token 转成 ID
ids = tokenizer.convert_tokens_to_ids(tokens)
print("Token IDs:", ids)

# 解码（还原为原文）
decoded = tokenizer.decode(ids)
print("Decoded:", decoded)

   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.0 MB 2.6 MB/s eta 0:00:05
   ----- ---------------------------------- 1.6/12.0 MB 3.1 MB/s eta 0:00:04
   ------- -------------------------------- 2.4/12.0 MB 3.4 MB/s eta 0:00:03
   ---------- ----------------------------- 3.1/12.0 MB 3.6 MB/s eta 0:00:03
   ------------- -------------------------- 3.9/12.0 MB 3.7 MB/s eta 0:00:03
   ---------------- ----------------------- 5.0/12.0 MB 3.7 MB/s eta 0:00:02
   ------------------ --------------------- 5.5/12.0 MB 3.7 MB/s eta 0:00:02
   --------------------- ------------------ 6.6/12.0 MB 3.7 MB/s eta 0:00:02
   ------------------------ --------------- 7.3/12.0 MB 3.8 MB/s eta 0:00:02
   --------------------------- ------------ 8.4/12.0 MB 3.9 MB/s eta 0:00:01
   ------------------------------- -------- 9.4/12.0 MB 3.9 MB/s eta 0:00:01
   ----------

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\29611\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\29611\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Tokens: ['natural', 'language', 'processing', 'is', 'fun', '!']
Token IDs: [3019, 2653, 6364, 2003, 4569, 999]
Decoded: natural language processing is fun!


#### WordPiece 是BPE的改进版本，同样是基于子词进行分词。但是不同于BPE的贪心的选择。
#### WordPiece 不是按照频率来合并而是按照最大化语言模型的似然估计来选最优子词合并。
#### 即在有限词汇表下，让文本的概率最大


### 过程
#### 1. 从字符开始
#### 2. 统计相邻子词的联合概率P(xy)
#### 3. 计算合并后的似然提升$ \frac{P(x)}{P(y)P(xy)} $
#### 4. 选择最大化预料总体似然子词进行合并
#### 5. 重复